In [1]:
%pip install scikit-surprise

     |████████████████████████████████| 771 kB 747 kB/s eta 0:00:01
     |████████████████████████████████| 301 kB 663 kB/s eta 0:00:01
     |████████████████████████████████| 29.6 MB 477 kB/s eta 0:00:01
Using legacy 'setup.py install' for scikit-surprise, since package 'wheel' is not installed.
    Running setup.py install for scikit-surprise ... done
You should consider upgrading via the '/Users/tgayanvitharanagamage/workdir/repos/templates/venv/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from surprise import Reader
import pandas as pd

# Hyper-parameter tuninng with Grid Search

## Movielens dataset

In [2]:
data = Dataset.load_builtin("ml-100k")
data

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/tgayanvitharanagamage/.surprise_data/ml-100k


In [9]:
print(dir(data))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'build_full_trainset', 'construct_testset', 'construct_trainset', 'has_been_split', 'load_builtin', 'load_from_df', 'load_from_file', 'load_from_folds', 'ratings_file', 'raw_ratings', 'read_ratings', 'reader']


In [5]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

In [6]:
gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [7]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

0.9442628957696252
{'sim_options': {'name': 'msd', 'min_support': 3, 'user_based': False}}


## Amazon dataset

In [2]:
reader = Reader(rating_scale=(1, 5))

In [3]:
ratings_beauty_df = pd.read_csv("ratings_Beauty.csv")
ratings_beauty_df.shape

(2023070, 4)

In [4]:
ratings_beauty_df = ratings_beauty_df.head(200000)
ratings_beauty_df

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200
...,...,...,...,...
199995,A029960025H0RFSRCHXUW,B000C1Z1YC,5.0,1384041600
199996,A34GZMKEI89MUR,B000C1Z1YC,5.0,1400025600
199997,A118JJ8W51L06I,B000C1Z1YC,2.0,1316649600
199998,A38KRDJP87P086,B000C1Z1YC,1.0,1403222400


In [5]:
# Loads Pandas dataframe
ratings_beauty = Dataset.load_from_df(ratings_beauty_df[["UserId", "ProductId", "Rating"]], reader)
ratings_beauty

In [6]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

In [ ]:
gs_ratings_beauty = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs_ratings_beauty.fit(ratings_beauty)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
print(gs_ratings_beauty.best_score["rmse"])
print(gs_ratings_beauty.best_params["rmse"])

#### Previous results with 100000 data
1.3217528548434327
{'sim_options': {'name': 'cosine', 'min_support': 4, 'user_based': False}}

# Prediction

## Amazon dataset

Use best parameters found by grid search to build the recommender model

In [20]:
# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "min_support": 4,
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [21]:
trainingSet = ratings_beauty.build_full_trainset()

In [22]:
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [26]:
prediction = algo.predict('A39HTATAQ9V7YF', '0558925278')
prediction.est

4.0